In [85]:
import pandas as pd
from matplotlib import pyplot as plt

from gensim.models.word2vec import Word2Vec
from os import path, mkdir
from multiprocessing import cpu_count

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, regexp_tokenize
from konlpy.tag import Kkma,Twitter
from collections import Counter

In [3]:
d = pd.read_excel('test.xlsx',sheet_name='sheet1')

In [5]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8035 entries, 0 to 8034
Data columns (total 13 columns):
연월      8035 non-null int64
전시제목    8035 non-null object
링크      8035 non-null object
전시작가    8035 non-null object
전시일정    8035 non-null object
초대일시    7629 non-null object
관람시간    8034 non-null object
전시장소    8034 non-null object
주소      8034 non-null object
연락처     8034 non-null object
홈페이지    7432 non-null object
아티클     8034 non-null object
텍스트     7703 non-null object
dtypes: int64(1), object(12)
memory usage: 878.8+ KB


In [13]:
d.dropna(subset=['텍스트'],inplace=True) #텍스트 없는 로우 드랍

In [16]:
sum(d.duplicated(subset=['텍스트'])) # 중복된 텍스트

20

In [18]:
toDropIndex = d[d.duplicated(subset=['텍스트'])].index

In [19]:
d.drop(index = toDropIndex, inplace=True)

In [20]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7683 entries, 0 to 8034
Data columns (total 13 columns):
연월      7683 non-null int64
전시제목    7683 non-null object
링크      7683 non-null object
전시작가    7683 non-null object
전시일정    7683 non-null object
초대일시    7292 non-null object
관람시간    7683 non-null object
전시장소    7683 non-null object
주소      7683 non-null object
연락처     7683 non-null object
홈페이지    7103 non-null object
아티클     7683 non-null object
텍스트     7683 non-null object
dtypes: int64(1), object(12)
memory usage: 840.3+ KB


In [21]:
corpus = ''
for txt in d.텍스트:
    if txt==None:
        print('None')
        continue
    corpus +=str(txt)

In [89]:
# kkma = Kkma()
# tokens = kkma.morphs(corpus) 

# twitter = Twitter()
# tokens = twitter.morphs(corpus)# 찢어서하자

In [24]:
## 문장 단위의 token을 생성 
sent_tokens = sent_tokenize(corpus)

In [25]:
len(sent_tokens)

183701

In [26]:
word_tokens = []
for sent in sent_tokens:
    words = word_tokenize(sent)
    word_tokens.extend(words)
    

In [27]:
len(word_tokens)

3487192

In [34]:
word_tokens = [word for word in word_tokens if len(word)>1] #1글자 초과 단어만 수집

In [35]:
len(word_tokens)

2818755

In [42]:
for w in word_tokens[:10]:
    print(w)

광주시립미술관
관장
황영성
2014년
광주시립미술관
갤러리GMA
번째
기획전으로
남도작가
12인


In [64]:
art2vec =Word2Vec(
    sg=1, #skip-gram
    workers=cpu_count(),
    size=100,
    min_count=2,
    window=10,
)

In [65]:
art2vec.build_vocab(sent_tokens)

In [69]:
art2vec.corpus_count

183701

In [72]:
art2vec.epochs

In [78]:
art2vec.train(sent_tokens,total_examples=art2vec.corpus_count,epochs=3)

(23237323, 43136931)

In [81]:
##모델을 저장하고 불러와서 다시 training시킬 수 있다
art2vec.save(r'''art2vec.w2v''')

In [88]:
# art2vec.wv.vocab.keys()

In [80]:
art2vec.wv.most_similar('광')

[('읍', 0.6338919401168823),
 ('▷', 0.580043613910675),
 ('북', 0.5710780620574951),
 ('옥', 0.5700820684432983),
 ('㈜', 0.5688358545303345),
 ('산', 0.5588215589523315),
 ('봉', 0.557214617729187),
 ('※', 0.5569085478782654),
 ('좡', 0.5554145574569702),
 ('쏜', 0.5476833581924438)]

In [82]:
art2vec.wv.most_similar(negative='광',)

[('形', 0.07049774378538132),
 ('視', 0.06646951287984848),
 ('?', 0.05360574275255203),
 ('工', 0.0509970597922802),
 ('的', 0.04029640555381775),
 ('識', 0.030930187553167343),
 ('態', 0.01797778159379959),
 ('.', 0.011267516762018204),
 ('엇', 0.00847981870174408),
 ('性', 0.004824429750442505)]